In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import os
import pandas as pd
import os
from tqdm.notebook import tqdm
from PIL import Image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 8
IMG_WIDTH = 224
IMG_HEIGHT = 224

In [3]:
target_files = ["1652875851.3497071", "1652875901.3107166", "1652876013.741493", "1652876206.2541456", "1652876485.8123376", "1652959186.4507334",
                "1652959347.972946", "1653042695.4914637", "1653042775.5213027", "1653043202.5073502", "1653043345.3415065", "1653043428.8546412", "1653043549.5187616"]

In [4]:
random_transforms = transforms.Compose([
    transforms.RandomRotation(5),
    transforms.RandomResizedCrop((IMG_WIDTH, IMG_HEIGHT), scale=(.9, 1), ratio=(1, 1)),
    transforms.ColorJitter(brightness=.2, contrast=.5,saturation=0.5,hue=0.5)
])

In [5]:
class ImageDataset(Dataset):
    def __init__(self, files):
        self.image_dirs = files
        self.targets = pd.DataFrame(columns=['path', 'forward', 'left'])
        for image_dir in self.image_dirs:
            def change_name(number:float) -> str:
                true_num = int(number)
                return f"{os.path.join(image_dir, f'{true_num:04}.jpg')}"
            frame = pd.read_csv(os.path.join('dataset', f'{image_dir}.csv'), names=['path', 'forward', 'left'])
            frame['path'] = frame['path'].apply(change_name)
            self.targets = pd.concat([self.targets, frame])
            self.paths:pd.Series[str] = self.targets.pop('path')
            break

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        image_path = self.paths[idx]
        target = torch.tensor(self.targets.iloc[idx].values,dtype=torch.float32)
        image = transforms.ToTensor()(Image.open(os.path.join('dataset', image_path)))
        image = image.to(device)
        target = target.to(device)
        return image, target


In [6]:
img_dataset = ImageDataset(target_files)

/tmp/ipykernel_16848/4171472145.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.targets = pd.concat([self.targets, frame])


In [7]:
train_dataset, val_dataset = random_split(img_dataset, [0.8, 0.2])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [8]:
class DrivingModel(nn.Sequential):
    def __init__(self):
        super(DrivingModel, self).__init__()
        self.input_shape = (1, 3, IMG_WIDTH, IMG_HEIGHT)
        
        self.append(nn.Conv2d(3, 16, kernel_size=3, padding=1))
        #self.append(nn.BatchNorm2d(16))
        self.append(nn.ReLU())
        #self.append(nn.Dropout2d(.3))
        self.append(nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.append(nn.Conv2d(16, 32, kernel_size=3, padding=1))
        #self.append(nn.BatchNorm2d(32))
        self.append(nn.ReLU())
        #self.append(nn.Dropout2d(.3))
        self.append(nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.append(nn.Conv2d(32, 64, kernel_size=3, padding=1))
        #self.append(nn.BatchNorm2d(64))
        self.append(nn.ReLU())
        #self.append(nn.Dropout2d(.3))
        self.append(nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.append(nn.Flatten())
        
        self.append(nn.Linear(64 * 28 * 28, 64))
        #self.append(nn.Dropout1d(.3))
        self.append(nn.ReLU())
        self.append(nn.Linear(64,2))

    def forward(self, x):
        x = x.view(-1, 3, 224, 224)
        for layer in self:
            x = layer(x)
        return x

In [9]:
model= DrivingModel().to(device)
model = torch.compile(model)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [10]:
num_epochs = 200
best_val_loss = None
counter = 10
for epoch in range(num_epochs):
    train_loss = 0
    model.train()
    for images, targets in tqdm(train_loader, desc="Training"):
        optimizer.zero_grad()
        images = random_transforms(images)
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss / len(train_loader)}")
    
    val_loss = 0
    model.eval()
    for images, targets in tqdm(val_loader, desc="Validation"):
        images = random_transforms(images)
        images = images.to(device)
        targets = targets.to(device)
        with torch.inference_mode():
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
    if best_val_loss is None:
        best_val_loss = val_loss
    elif best_val_loss < val_loss:
        counter -=1
    else:
        best_val_loss = val_loss
        counter = 10
    if not counter:
        break
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss / len(val_loader)}")
torch.save(model, 'model.pth')

Training:   0%|          | 0/19 [00:00<?, ?it/s]

W0518 12:10:30.418000 16848 torch/_inductor/utils.py:1250] [0/0] Not enough SMs to use max_autotune_gemm mode


Epoch 1/200, Training Loss: 0.3821396971807668


Validation:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/200, Validation Loss: 0.1484855130314827


...

Validation:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 41/200, Validation Loss: 0.096139794588089


Training:   0%|          | 0/19 [00:00<?, ?it/s]

Epoch 42/200, Training Loss: 0.08916492946445942


Validation:   0%|          | 0/5 [00:00<?, ?it/s]